## Segment Retinal Atrophy

### UNET model

In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split

In [2]:
def load_data(image_dir, mask_dir, image_size):
    images = []
    masks = []
    
    # Loop through all image files
    for filename in os.listdir(image_dir):
        if filename.endswith(".jpg"):
            # Get the base name (e.g., 'H0001' from 'H0001.jpg')
            base_name = os.path.splitext(filename)[0]
            
            # Construct the corresponding mask filename
            mask_filename = f"{base_name}.bmp"
            
            # Check if the corresponding mask exists
            img_path = os.path.join(image_dir, filename)
            mask_path = os.path.join(mask_dir, mask_filename)
        
            if os.path.exists(mask_path):
                # Load and resize the image and mask
                img = load_img(img_path, target_size=image_size)
                mask = load_img(mask_path, target_size=image_size, color_mode="grayscale")
                
                # Convert to numpy arrays
                img_array = img_to_array(img) / 255.0  # Normalize the image between 0 and 1
                mask_array = img_to_array(mask) / 255.0  # Mask is binary
                
                # Append the arrays to the lists
                images.append(img_array)
                masks.append(mask_array)
            else:
                print(f"Skipping {filename} as no corresponding mask was found.")
    
    # Convert the lists to numpy arrays
    return np.array(images), np.array(masks)

def unet_model(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)
    
    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Bottleneck
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    
    # Decoder
    up1 = layers.UpSampling2D(size=(2, 2))(conv3)
    up1 = layers.concatenate([up1, conv2], axis=-1)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(up1)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)
    
    up2 = layers.UpSampling2D(size=(2, 2))(conv4)
    up2 = layers.concatenate([up2, conv1], axis=-1)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(up2)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)
    
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv5)  # Sigmoid for binary classification
    
    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

In [3]:
model = unet_model(input_size=(256, 256, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

2024-09-25 23:22:59.129998: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-25 23:22:59.130269: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [5]:
# Directories for images and masks
image_dir = 'path/to/images'
mask_dir = 'path/to/atrophy_masks'

# Image size (height, width, channels)
image_size = (256, 256)

all_images, all_masks = load_data(image_dir, mask_dir, image_size)

# Split into training and test sets (80% train, 20% test)
train_images, test_images, train_masks, test_masks = train_test_split(
    all_images, all_masks, test_size=0.2, random_state=42
)

# split train set into training and validation sets (90% train, 10% validation)
train_images, val_images, train_masks, val_masks = train_test_split(
    train_images, train_masks, test_size=0.1, random_state=42
)

Skipping N0010.jpg as no corresponding mask was found.
Skipping T0042.jpg as no corresponding mask was found.
Skipping V0379.jpg as no corresponding mask was found.
Skipping V0345.jpg as no corresponding mask was found.
Skipping T0254.jpg as no corresponding mask was found.
Skipping V0190.jpg as no corresponding mask was found.
Skipping V0147.jpg as no corresponding mask was found.
Skipping T0283.jpg as no corresponding mask was found.
Skipping P0076.jpg as no corresponding mask was found.
Skipping T0334.jpg as no corresponding mask was found.
Skipping N0158.jpg as no corresponding mask was found.
Skipping V0231.jpg as no corresponding mask was found.
Skipping V0225.jpg as no corresponding mask was found.
Skipping V0032.jpg as no corresponding mask was found.
Skipping T0335.jpg as no corresponding mask was found.
Skipping T0309.jpg as no corresponding mask was found.
Skipping T0296.jpg as no corresponding mask was found.
Skipping T0269.jpg as no corresponding mask was found.
Skipping V

In [6]:
len(train_images), len(val_images), len(test_images)

(626, 70, 174)

In [7]:
history = model.fit(train_images, train_masks, epochs=50,
                    batch_size=16, verbose=1, steps_per_epoch=10,
                    validation_data=(val_images, val_masks))

Epoch 1/50


2024-09-25 23:23:26.421519: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-09-25 23:23:26.831625: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - ETA: 0s - loss: 0.4589 - accuracy: 0.8660

2024-09-25 23:23:38.460250: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 15s 1s/step - loss: 0.4589 - accuracy: 0.8660 - val_loss: 0.3182 - val_accuracy: 0.9037
Epoch 2/50
10/10 [==============================] - 13s 1s/step - loss: 0.3256 - accuracy: 0.8905 - val_loss: 0.2524 - val_accuracy: 0.9037
Epoch 3/50
10/10 [==============================] - 13s 1s/step - loss: 0.2780 - accuracy: 0.8872 - val_loss: 0.2800 - val_accuracy: 0.9037
Epoch 4/50
10/10 [==============================] - 12s 1s/step - loss: 0.2491 - accuracy: 0.8882 - val_loss: 0.2855 - val_accuracy: 0.9037
Epoch 5/50
10/10 [==============================] - 12s 1s/step - loss: 0.2399 - accuracy: 0.9027 - val_loss: 0.2483 - val_accuracy: 0.9037
Epoch 6/50
10/10 [==============================] - 12s 1s/step - loss: 0.2395 - accuracy: 0.9047 - val_loss: 0.2170 - val_accuracy: 0.9158
Epoch 7/50
10/10 [==============================] - 12s 1s/step - loss: 0.2160 - accuracy: 0.9247 - val_loss: 0.2330 - val_accuracy: 0.9032
Epoch 8/50
10/10 [=============

In [8]:
scores = model.evaluate(val_images, val_masks)
print("Validation Loss: %.2f" % scores[0])
print("Validation Accuracy: %.2f%%" % (scores[1] * 100))

3/3 [==============================] - 2s 544ms/step - loss: 0.1359 - accuracy: 0.9474
Validation Loss: 0.14
Validation Accuracy: 94.74%


In [9]:
scores = model.evaluate(test_images, test_masks)
print("Test Loss: %.2f" % scores[0])
print("Test Accuracy: %.2f%%" % (scores[1] * 100))

6/6 [==============================] - 5s 765ms/step - loss: 0.1513 - accuracy: 0.9418
Test Loss: 0.15
Test Accuracy: 94.18%


In [11]:
# Save the model
model_json = model.to_json()
with open('../features/features_segmentation.json', 'w') as json_file:
    json_file.write(model_json)

# Saving the model and weights
model.save_weights('../features_segmentation/Atrophy.weights.h5')